# Q1

In [20]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

In [13]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Get weather data for a specific city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city to query the weather data"
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

## Test it

In [14]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# Cargar .env
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [15]:
import chat_assistant

tools = chat_assistant.Tools()
tools.add_tool(get_weather, get_weather_tool)

In [16]:
tools.get_tools()

[{'type': 'function',
  'name': 'get_weather',
  'description': 'Get weather data for a specific city',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'The city to query the weather data'}},
   'required': ['city'],
   'additionalProperties': False}}]

In [17]:
openai_client = OpenAI(api_key=api_key)

In [18]:

developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=openai_client
)

In [19]:
chat.run()

Chat ended.
